In [ ]:
import dgl
import torch
import torch.nn.functional as F
from dgl.nn.pytorch import GATConv
from sklearn.metrics import roc_auc_score
import numpy as np
import torch.backends.cudnn as cudnn
import random
import pickle as pkl


In [ ]:
# read G_train_dgl, G_val_dgl, G_test_dgl, use pickle to read
with open('G_train_dgl.gpickle', 'rb') as f:
    G_train_dgl = pkl.load(f)
    
with open('G_val_dgl.gpickle', 'rb') as f:
    G_val_dgl = pkl.load(f)
    
with open('G_test_dgl.gpickle', 'rb') as f:
    G_test_dgl = pkl.load(f)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
import copy
import random
# Set the random seed, a randamly selected number
seed = random.randint(0, 1000)
print(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

class GATModel(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_heads):
        super().__init__()
        self.conv1 = GATConv(in_dim, hidden_dim, num_heads=num_heads)
        self.conv2 = GATConv(hidden_dim * num_heads, out_dim, num_heads)
    
    def forward(self, g, h):
        h = self.conv1(g, h).flatten(1)
        h = F.elu(h)
        h = self.conv2(g, h).mean(1)
        return h

# Get the number of input features
in_feats = G_train_dgl.ndata['features'].shape[1]

# Define the model hyperparameters
hidden_size = 128
out_feats = 2  # Assuming binary classification
num_heads = 3

In [ ]:
# import macro-F1, accuracy from sklearn
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from sklearn.metrics import f1_score, accuracy_score
import copy
import random
# Set the random seed, a randamly selected number
seed = random.randint(0, 1000)
print(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

class GATModel(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_heads, dropout_rate=0.1):
        super().__init__()
        self.conv1 = GATConv(in_dim, hidden_dim, num_heads=num_heads)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.conv2 = GATConv(hidden_dim * num_heads, hidden_dim, num_heads)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.conv3 = GATConv(hidden_dim * num_heads, out_dim, num_heads)
    
    def forward(self, g, h):
        h = self.conv1(g, h).flatten(1)
        h = F.elu(self.dropout1(h))
        h = self.conv2(g, h).flatten(1)
        h = F.elu(self.dropout2(h))
        h = self.conv3(g, h).mean(1)
        return h


# Get the number of input features
in_feats = G_train_dgl.ndata['features'].shape[1]

# Define the model hyperparameters
hidden_size = 128
out_feats = 2  # Assuming binary classification
num_heads = 3

# Create the GCN model
model = GATModel(in_feats, hidden_size, out_feats, num_heads)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()
best_val_loss = float('inf')
best_model = None
num_epochs = 200
patience = 20

# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    labels = G_train_dgl.ndata['label'].squeeze()
    features = G_train_dgl.ndata['features']

    # Select indices of 0 and 1 labels
    zero_indices = torch.where(labels == 0)[0]
    one_indices = torch.where(labels == 1)[0]
    
    # Get the minimum count between 0 and 1 labels
    min_count = min(zero_indices.shape[0], one_indices.shape[0])
    
    # Randomly select 'min_count' indices from zero_indices and one_indices each
    selected_zero_indices = zero_indices[torch.randperm(zero_indices.shape[0])[:min_count]]
    selected_one_indices = one_indices[torch.randperm(one_indices.shape[0])[:min_count]]

    # Combine the selected indices
    selected_indices = torch.cat((selected_zero_indices, selected_one_indices))

    # Shuffle the selected indices
    selected_indices = selected_indices[torch.randperm(selected_indices.shape[0])]

    # Create a subgraph from the selected indices
    subgraph = dgl.node_subgraph(G_train_dgl, selected_indices)

    # Get the selected features and labels
    selected_features = subgraph.ndata['features']
    selected_labels = subgraph.ndata['label'].squeeze()

    # Forward pass and compute the loss
    logits = model(subgraph, selected_features)
    labels = F.one_hot(selected_labels, num_classes=out_feats).float()
    loss = criterion(logits, labels)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
         # also 1-to-1 mapping for validation
        logits = model(G_val_dgl, G_val_dgl.ndata['features'])
        labels = F.one_hot(G_val_dgl.ndata['label'].squeeze(), num_classes=out_feats).float()
        val_loss = criterion(logits, labels)

    # Add early stopping based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)
        torch.save(model.state_dict(), 'best_model.pt')  # Save the best model

    print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {loss:.4f}, Validation Loss: {val_loss:.4f}")

best_model.eval()
with torch.no_grad():
    # Create balanced testing set
    labels = G_test_dgl.ndata['label'].squeeze()

    # Select indices of 0 and 1 labels
    zero_indices = torch.where(labels == 0)[0]
    one_indices = torch.where(labels == 1)[0]

    # Get the minimum count between 0 and 1 labels
    min_count = min(zero_indices.shape[0], one_indices.shape[0])

    # Randomly select 'min_count' indices from zero_indices and one_indices each
    selected_zero_indices = zero_indices[torch.randperm(zero_indices.shape[0])[:min_count]]
    selected_one_indices = one_indices[torch.randperm(one_indices.shape[0])[:min_count]]

    # Combine the selected indices
    selected_indices = torch.cat((selected_zero_indices, selected_one_indices))

    # Shuffle the selected indices
    selected_indices = selected_indices[torch.randperm(selected_indices.shape[0])]

    # Create a subgraph from the selected indices
    subgraph = dgl.node_subgraph(G_test_dgl, selected_indices)

    # Get the selected features and labels
    selected_features = subgraph.ndata['features']
    ground_truth = subgraph.ndata['label'].squeeze()

    # Testing
    logits = best_model(subgraph, selected_features)
    _, predicted_labels = torch.max(logits, 1)

    # Calculate additional evaluation metrics for testing
    predicted_probs = F.softmax(logits, dim=1)[:, 1]
    auc = roc_auc_score(ground_truth.detach().numpy(), predicted_probs.detach().numpy())
    f1 = f1_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy())
    precision = precision_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy())
    recall = recall_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy())

    print(f"AUC: {auc:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from sklearn.metrics import f1_score, accuracy_score
import copy
import random

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


# normalize features in G_train_dgl
train_feats = G_train_dgl.ndata['features']
scaler.fit(train_feats)

In [ ]:
# then transform train_feats, val_feats, test_feats
train_feats_normalized = scaler.transform(train_feats)

In [ ]:
# do the same thing for val_feats and test_feats
val_feats = G_val_dgl.ndata['features']
val_feats_normalized = scaler.transform(val_feats)

test_feats = G_test_dgl.ndata['features']
test_feats_normalized = scaler.transform(test_feats)

In [ ]:
# write back to G_train_dgl, G_val_dgl, G_test_dgl
G_train_dgl.ndata['normalized_features'] = torch.FloatTensor(train_feats_normalized)
G_val_dgl.ndata['normalized_features'] = torch.FloatTensor(val_feats_normalized)
G_test_dgl.ndata['normalized_features'] = torch.FloatTensor(test_feats_normalized)

In [ ]:
# print normalized_features in G_train_dgl
print(G_train_dgl.ndata['normalized_features'])

In [ ]:
# store back to pickle
with open('G_train_dgl.gpickle', 'wb') as f:
    pkl.dump(G_train_dgl, f)
    
with open('G_val_dgl.gpickle', 'wb') as f:
    pkl.dump(G_val_dgl, f)
    
with open('G_test_dgl.gpickle', 'wb') as f:
    pkl.dump(G_test_dgl, f)

In [ ]:
for i in range(10):
    # Set the random seed, a randamly selected number
    seed = random.randint(0, 1000)
    print(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    # Create the GCN model
    model = GATModel(8, 128, 2, 3)
    # Define the optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.BCEWithLogitsLoss()
    best_val_loss = float('inf')
    best_model = None
    num_epochs = 200
    patience = 20

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        labels = G_train_dgl.ndata['label'].squeeze()
        features = G_train_dgl.ndata['normalized_features']

        # Select indices of 0 and 1 labels
        zero_indices = torch.where(labels == 0)[0]
        one_indices = torch.where(labels == 1)[0]
        
        # Get the minimum count between 0 and 1 labels
        min_count = min(zero_indices.shape[0], one_indices.shape[0])
        
        # Randomly select 'min_count' indices from zero_indices and one_indices each
        selected_zero_indices = zero_indices[torch.randperm(zero_indices.shape[0])[:min_count]]
        selected_one_indices = one_indices[torch.randperm(one_indices.shape[0])[:min_count]]

        # Combine the selected indices
        selected_indices = torch.cat((selected_zero_indices, selected_one_indices))

        # Shuffle the selected indices
        selected_indices = selected_indices[torch.randperm(selected_indices.shape[0])]

        # Create a subgraph from the selected indices
        subgraph = dgl.node_subgraph(G_train_dgl, selected_indices)

        # Get the selected features and labels
        selected_features = subgraph.ndata['normalized_features']
        selected_labels = subgraph.ndata['label'].squeeze()

        # Forward pass and compute the loss
        logits = model(subgraph, selected_features)
        labels = F.one_hot(selected_labels, num_classes=out_feats).float()
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            # Create balanced validation set
            labels = G_val_dgl.ndata['label'].squeeze()

            # Select indices of 0 and 1 labels
            zero_indices = torch.where(labels == 0)[0]
            one_indices = torch.where(labels == 1)[0]

            # Get the minimum count between 0 and 1 labels
            min_count = min(zero_indices.shape[0], one_indices.shape[0])

            # Randomly select 'min_count' indices from zero_indices and one_indices each
            selected_zero_indices = zero_indices[torch.randperm(zero_indices.shape[0])[:min_count]]
            selected_one_indices = one_indices[torch.randperm(one_indices.shape[0])[:min_count]]

            # Combine the selected indices
            selected_indices = torch.cat((selected_zero_indices, selected_one_indices))

            # Shuffle the selected indices
            selected_indices = selected_indices[torch.randperm(selected_indices.shape[0])]

            # Create a subgraph from the selected indices
            subgraph = dgl.node_subgraph(G_val_dgl, selected_indices)

            # Get the selected features and labels
            selected_features = subgraph.ndata['normalized_features']
            selected_labels = subgraph.ndata['label'].squeeze()

            # Validation
            logits = model(subgraph, selected_features)
            labels = F.one_hot(selected_labels, num_classes=out_feats).float()
            val_loss = criterion(logits, labels)
            
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = copy.deepcopy(model)
            torch.save(model.state_dict(), 'best_model.pt')
        print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {loss:.4f}, Validation Loss: {val_loss:.4f}")

    best_model.eval()
    with torch.no_grad():
        # Create balanced testing set
        labels = G_test_dgl.ndata['label'].squeeze()

        # Select indices of 0 and 1 labels
        zero_indices = torch.where(labels == 0)[0]
        one_indices = torch.where(labels == 1)[0]

        # Get the minimum count between 0 and 1 labels
        min_count = min(zero_indices.shape[0], one_indices.shape[0])

        # Randomly select 'min_count' indices from zero_indices and one_indices each
        selected_zero_indices = zero_indices[torch.randperm(zero_indices.shape[0])[:min_count]]
        selected_one_indices = one_indices[torch.randperm(one_indices.shape[0])[:min_count]]

        # Combine the selected indices
        selected_indices = torch.cat((selected_zero_indices, selected_one_indices))

        # Shuffle the selected indices
        selected_indices = selected_indices[torch.randperm(selected_indices.shape[0])]

        # Create a subgraph from the selected indices
        subgraph = dgl.node_subgraph(G_test_dgl, selected_indices)

        # Get the selected features and labels
        selected_features = subgraph.ndata['normalized_features']
        ground_truth = subgraph.ndata['label'].squeeze()

        # Testing
        logits = best_model(subgraph, selected_features)
        _, predicted_labels = torch.max(logits, 1)

        # Calculate additional evaluation metrics for testing
        predicted_probs = F.softmax(logits, dim=1)[:, 1]
        # adjust the threshold to 0.5
        predicted_labels = (predicted_probs > 0.4).float()
        auc = roc_auc_score(ground_truth.detach().numpy(), predicted_probs.detach().numpy())
        f1 = f1_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy())
        precision = precision_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy())
        recall = recall_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy())
        accuracy = accuracy_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy())
        macro_f1 = f1_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy(), average='macro')
        macro_precision = precision_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy(), average='macro')
        macro_recall = recall_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy(), average='macro')
        # store results in a txt file
        with open("GAT_wo_results.txt", "a") as f:
            # need to write random seed, validation loss, test loss, auc, f1, precision, recall
            f.write(f"Random seed: {seed}, Epoch: {epoch + 1}/{num_epochs}, Loss: {loss:.4f}, Validation Loss: {val_loss:.4f}, AUC: {auc:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, Macro-F1: {macro_f1:.4f}, Macro-Precision: {macro_precision:.4f}, Macro-recall: {macro_recall:.4f}\n")
        print(f"AUC: {auc:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, Macro-F1: {macro_f1:.4f}, Macro-Precision: {macro_precision:.4f}, Macro-recall: {macro_recall:.4f}\n")
